<b><i> analysis </i></b>

Created by Eduardo Alastrué de Asenjo

- Purpose: Analysis & figures ERL paper "European heat extremes under net-zero emissions"
- Data: Model data from ACCESS-ESM-1.5 & reanalysis data from ERA5
- Packages: use xarray with dask to handle data, regionmask for selecting European regions, and matplotlib and cartopy for plotting
- Comments: It originally runs on Gadi, NCI's HPC, using conda env:analysis3 kernel

# Initialisation

Load modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import matplotlib.path as mpath
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)},font_scale=1.5)
sns.set_style("white")
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import subprocess
import math as math
import netCDF4 as nc
from scipy import stats
from random import choices
import copy
import xarray as xr
import pandas as pd
import datetime
import cftime
from tqdm import tqdm
import glob
import pickle
import cdo
import statsmodels.api as sm 
import warnings
import pylab as py 
import regionmask
import xclim as xc
import xskillscore  
import itertools
import pyam # IPCC colors
colors = pyam.plotting.PYAM_COLORS
import logging # supress flox info messages 
logging.getLogger('flox').setLevel(logging.WARNING)
from IPython.core.magic import register_cell_magic # Custom magic command for cell running time 
import time
@register_cell_magic
def timing(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    print(f"Cell run at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, Exec. time: {end_time - start_time:.2f} seconds")

<IPython.core.display.Javascript object>

load functions

In [2]:
def weighted_mon_to_year_mean(ds, var):
    """
    weight by days in each month when doing annual mean from monthly values in xarray
    taken from https://ncar.github.io/esds/posts/2021/yearly-averages-xarray/
    """
    month_length = ds.time.dt.days_in_month # Determine the month length

    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum() # Calculate the weights

    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0) # Make sure the weights in each year add up to 1

    obs = ds[var] # Subset our dataset for our variable

    cond = obs.isnull() # Setup our masking for nan values
    ones = xr.where(cond, 0.0, 1.0)
    
    obs_sum = (obs * wgts).resample(time="YS").sum(dim="time") # Calculate the numerator
    ones_out = (ones * wgts).resample(time="YS").sum(dim="time") # Calculate the denominator

    return obs_sum / ones_out # Return the weighted average

In [3]:
def weighted_area_lat(ds):
    """
    Calculate the area-weighted temperature over its domain. 
    In a regular latitude/ longitude grid the grid cell area decreases towards the pole.
    We can use the cosine of the latitude as proxy for the grid cell area.
    IMPORTANT: after applying function, do .mean('lat') before 'lon'
    Taken from https://docs.xarray.dev/en/stable/examples/area_weighted_temperature.html
    """
    weights = np.cos(np.deg2rad(ds.lat))
    weights.name = "weights"

    return ds.weighted(weights)

In [4]:
def concat_ensmem(ds): 
    '''
    Function to concatenate all ensemble members across dimension 'realiz'. Original times are dropped
    '''
    data_list = []
    for i in ds.realiz:
        da = ds.sel(realiz=i)
        data_list.append(da)
    ds_concat = xr.concat(data_list, dim='time').dropna(dim='time', how='all')
    ds_concat['time'] = range(len(ds_concat['time']))     # reset the 'time' coordinate to make it continuous
    return ds_concat

start dask cluster

In [ ]:
# local cluster
from dask.distributed import Client
client = Client(memory_limit=None, threads_per_worker=1, n_workers=18)

# Load data 

## tas (mon)

stab runs

In [ ]:
tas_yr_sta = xr.open_mfdataset("/scratch/p66/ea1694/data/stab_runs/tas_yr_sta.nc", use_cftime=True)

In [ ]:
# stab runs
mask_stab_eur = regionmask.defined_regions.ar6.land[16, 17, 18, 19].mask(tas_yr_sta)  # Europe
mask_stab_neu = mask_stab_eur == 16
mask_stab_wce = mask_stab_eur == 17
mask_stab_eeu = mask_stab_eur == 18
mask_stab_med = mask_stab_eur == 19

mask_stab_land = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(tas_yr_sta) # Natural earth masks
mask_stab_ocean = regionmask.defined_regions.natural_earth_v5_1_2.ocean_basins_50.mask(tas_yr_sta)

In [ ]:
tas_yr_sta_eur = tas_yr_sta.where(mask_stab_eur>15, drop=True).where(mask_stab_land==0, drop=True)

historical & scenarios

In [ ]:
path_ssp = {
    'ssp585': '/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp585/'
}

path_hist = {
    'historical': '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/'
}

In [ ]:
%%timing
# historical
array = []
for i in range(1,41):
    rea = "r"+str(i)+"i1p1f1"
    if os.path.isdir(f'{path_hist["historical"]}/{rea}'):
        infiles = glob.glob(f'{path_hist["historical"]}/{rea}/Amon/tas/gn/latest/*.nc', recursive=True)
        array.append(xr.open_mfdataset(infiles, use_cftime=True, data_vars="minimal", coords="minimal", chunks={'time': 1000}, parallel=True, compat="override").assign_coords(realiz=rea)) 
tas_mon_hist = xr.concat(array, dim='realiz')

In [ ]:
tas_yr_hist = weighted_mon_to_year_mean(tas_mon_hist, 'tas').compute()

In [ ]:
tas_yr_hist_first50y = tas_yr_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
tasref_50yrhist_yr = weighted_area_lat(tas_yr_hist_first50y).mean('lat').mean('lon').mean('time').mean('realiz').compute()

In [ ]:
%%timing
# ssp585
array = []
for i in range(1,41):
    rea = "r"+str(i)+"i1p1f1"
    if os.path.isdir(f'{path_ssp["ssp585"]}/{rea}'):
        infiles = glob.glob(f'{path_ssp["ssp585"]}/{rea}/Amon/tas/gn/latest/*.nc', recursive=True)
        array.append(xr.open_mfdataset(infiles, use_cftime=True, data_vars="minimal", coords="minimal", chunks={'time': 1000}, parallel=True, compat="override").assign_coords(realiz=rea)) 
tas_mon_ssp585 = xr.concat(array, dim='realiz')

In [ ]:
tas_yr_ssp585 = weighted_mon_to_year_mean(tas_mon_ssp585, 'tas').compute()

## TXx

load monthly and yearly files

In [ ]:
ind_paths = {
    'stab':      "/scratch/p66/txz599/processed_output/data/stab_runs/indices/",
    'scenarios': "/scratch/p66/txz599/processed_output/data/ACCESS-ESM1.5/scenarios/indices/",
    'era':       "/scratch/p66/txz599/processed_output/data/era5/indices/"
}

txx_mon_stab   = xr.open_mfdataset(f"{ind_paths['stab']}txx_mon.nc", chunks={'time': 120}, use_cftime=True)
txx_mon_hist   = xr.open_mfdataset(f"{ind_paths['scenarios']}txx_mon_hist.nc", chunks={'time': 120}, use_cftime=True)
txx_mon_ssp585 = xr.open_mfdataset(f"{ind_paths['scenarios']}txx_mon_ssp585.nc", chunks={'time': 120}, use_cftime=True)
txx_mon_era    = xr.open_mfdataset(f"{ind_paths['era']}txx_mon_*.nc", use_cftime=True)
txx_year_stab   = xr.open_mfdataset(f"{ind_paths['stab']}txx_year.nc", chunks={'time': 120}, use_cftime=True)
txx_year_hist   = xr.open_mfdataset(f"{ind_paths['scenarios']}txx_year_hist.nc", chunks={'time': 120}, use_cftime=True)
txx_year_ssp585 = xr.open_mfdataset(f"{ind_paths['scenarios']}txx_year_ssp585.nc", chunks={'time': 120}, use_cftime=True)
txx_year_era    = xr.open_mfdataset(f"{ind_paths['era']}txx_year_*.nc", use_cftime=True)

In [ ]:
txx_year_stab = txx_year_stab.compute()
txx_year_hist = txx_year_hist.compute()
txx_year_ssp585 = txx_year_ssp585.compute()
txx_year_era = txx_year_era.compute()

In [ ]:
txx_mon_stab = txx_mon_stab.compute()
txx_mon_hist = txx_mon_hist.compute()
txx_mon_ssp585 = txx_mon_ssp585.compute()
txx_mon_era = txx_mon_era.compute()

European masks

In [ ]:
# stab runs mask
mask_stab_eur = regionmask.defined_regions.ar6.land[16, 17, 18, 19].mask(txx_mon_stab)  # Europe
mask_stab_neu = mask_stab_eur == 16
mask_stab_wce = mask_stab_eur == 17
mask_stab_eeu = mask_stab_eur == 18
mask_stab_med = mask_stab_eur == 19
masks_stab_eur_dict = {'NEU': mask_stab_neu, 'WCE': mask_stab_wce, 'EEU': mask_stab_eeu, 'MED': mask_stab_med}

mask_stab_land = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(txx_mon_stab) # Natural earth masks
mask_stab_ocean = regionmask.defined_regions.natural_earth_v5_1_2.ocean_basins_50.mask(txx_mon_stab)

# cmip6 mask
mask_scen_eur = regionmask.defined_regions.ar6.land[16, 17, 18, 19].mask(txx_mon_hist) # Europe
mask_scen_neu = mask_scen_eur == 16
mask_scen_wce = mask_scen_eur == 17
mask_scen_eeu = mask_scen_eur == 18
mask_scen_med = mask_scen_eur == 19
masks_scen_eur_dict = {'NEU': mask_scen_neu, 'WCE': mask_scen_wce, 'EEU': mask_scen_eeu, 'MED': mask_scen_med}

mask_scen_land = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(txx_mon_hist) # Natural earth masks
mask_scen_ocean = regionmask.defined_regions.natural_earth_v5_1_2.ocean_basins_50.mask(txx_mon_hist)

# era5 mask
mask_era_eur = regionmask.defined_regions.ar6.land[16, 17, 18, 19].mask(txx_year_era) # Europe
mask_era_neu = mask_era_eur == 16
mask_era_wce = mask_era_eur == 17
mask_era_eeu = mask_era_eur == 18
mask_era_med = mask_era_eur == 19
masks_era_eur_dict = {'NEU': mask_era_neu, 'WCE': mask_era_wce, 'EEU': mask_era_eeu, 'MED': mask_era_med}

mask_era_land = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(txx_year_era) # Natural earth masks
mask_era_ocean = regionmask.defined_regions.natural_earth_v5_1_2.ocean_basins_50.mask(txx_year_era)

In [ ]:
txx_year_stab_eur = txx_year_stab.where(mask_stab_eur>15, drop=True).where(mask_stab_land==0, drop=True)
txx_year_hist_eur = txx_year_hist.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
txx_year_ssp585_eur = txx_year_ssp585.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
txx_year_era_eur = txx_year_era.where(mask_era_eur>15, drop=True).where(mask_era_land==0, drop=True)

In [ ]:
txx_mon_stab_eur = txx_mon_stab.where(mask_stab_eur>15, drop=True).where(mask_stab_land==0, drop=True)
txx_mon_hist_eur = txx_mon_hist.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
txx_mon_ssp585_eur = txx_mon_ssp585.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
txx_mon_era_eur = txx_mon_era.where(mask_era_eur>15, drop=True).where(mask_era_land==0, drop=True)

## TX90p

In [ ]:
path = "/scratch/p66/ea1694/data/stab_runs/indices/tx90p/"
array = []
for exp in [2030,2035,2040,2045,2050,2055,2060]:
    array.append(xr.open_mfdataset(path+f"tx90p_yr_NZ{exp}_*.nc", use_cftime=True).assign_coords(exp=exp))
tx90p_yr_sta = xr.concat(array, dim='exp').rename({"__xarray_dataarray_variable__": "TX90p"})
tx90p_yr_sta = tx90p_yr_sta.sel(time=~tx90p_yr_sta.indexes['time'].duplicated()) # remove dates that are duplicated

array = []
for i in range(1,41):
    rea = "r"+str(i)+"i1p1f1"
    array.append(xr.open_mfdataset(path+f"tx90p_yr_hist_{rea}.nc", use_cftime=True).assign_coords(realiz=rea))
tx90p_yr_hist = xr.concat(array, dim='realiz').rename({"__xarray_dataarray_variable__": "TX90p"})

array = []
for i in range(1,41):
    rea = "r"+str(i)+"i1p1f1"
    array.append(xr.open_mfdataset(path+f"tx90p_yr_ssp585_{rea}.nc", use_cftime=True).assign_coords(realiz=rea))
tx90p_yr_ssp585 = xr.concat(array, dim='realiz').rename({"__xarray_dataarray_variable__": "TX90p"})

tx90p_yr_era5 = xr.open_mfdataset(f"/scratch/p66/ea1694/data/stab_runs/indices/tx90p/tx90p_yr_era_*.nc", use_cftime=True).rename(
    {"__xarray_dataarray_variable__": "TX90p"})

In [ ]:
tx90p_yr_sta = tx90p_yr_sta.compute()
tx90p_yr_hist = tx90p_yr_hist.compute()
tx90p_yr_ssp585 = tx90p_yr_ssp585.compute()
tx90p_yr_era5 = tx90p_yr_era5.compute()

In [ ]:
tx90p_yr_sta_eur = tx90p_yr_sta.where(mask_stab_eur>15, drop=True).where(mask_stab_land==0, drop=True)
tx90p_yr_hist_eur = tx90p_yr_hist.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
tx90p_yr_ssp585_eur = tx90p_yr_ssp585.where(mask_scen_eur>15, drop=True).where(mask_scen_land==0, drop=True)
tx90p_yr_era5_eur = tx90p_yr_era5.where(mask_era_eur>15, drop=True).where(mask_era_land==0, drop=True)

tx90p_yr_sta_eur = tx90p_yr_sta_eur.compute()
tx90p_yr_hist_eur = tx90p_yr_hist_eur.compute()
tx90p_yr_ssp585_eur = tx90p_yr_ssp585_eur.compute()
tx90p_yr_era5_eur = tx90p_yr_era5_eur.compute()

# Main figures

## ERA5 spatial rank (1)

In [ ]:
def categorize(rank, total_years):
    percentile = (rank - 1) / (total_years - 1)
    if percentile == 0.0:
        return 1  # Coldest
    elif percentile <= 0.05:
        return 2  # 5% Coldest
    elif percentile <= 0.10:
        return 3  # 10% Coldest
    elif percentile <= 0.25:
        return 4  # 25% Coldest
    elif percentile <= 0.75:
        return 5  # Near Normal
    elif percentile <= 0.90:
        return 6  # 25% Warmest
    elif percentile <= 0.95:
        return 7  # 10% Warmest
    elif percentile < 1.0:
        return 8  # 5% Warmest
    elif percentile == 1.0:
        return 9 # Warmest
    else:
        return 5  # nans in white

In [ ]:
fig = plt.figure(figsize=(14, 16))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.2)
levels = np.arange(1, 10)
cmap = plt.get_cmap('RdBu_r', len(levels)) 
cmap.set_bad(color='white')
norm = mcolors.BoundaryNorm(boundaries=np.arange(0.5, 9.5 + 1), ncolors=cmap.N)

ax1 = fig.add_subplot(gs[0:2, 0:2], projection = ccrs.PlateCarree())
yr = weighted_area_lat(txx_year_era_eur).mean('lat').mean('lon').tasmax.idxmax('time').dt.year.values
txx_up_to_year = txx_year_era_eur.tasmax.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(yr, 12, 31)))
ranks = txx_up_to_year.rank(dim='time')
rank_given_year = ranks.sel(time=str(yr))
total_years = txx_up_to_year.sizes['time']

# Apply categorization function
categories = xr.where(rank_given_year.isnull(), np.nan, rank_given_year)
categories = xr.apply_ufunc(categorize, rank_given_year, total_years,
                            vectorize=True, dask='parallelized', output_dtypes=[int])

im = categories.plot(ax=ax1, cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
ax1.set_title(f'$\mathbf{{EUROPE}}$ ERA5 TXx record, $\mathbf{{{yr}}}$')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.coastlines()
ax1.set_extent([-10.5, 60.5, 29.5, 73], crs=ccrs.PlateCarree())
ax1.grid(False)
ax1.text(-0.02, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')


ax2 = fig.add_subplot(gs[2, 0], projection = ccrs.PlateCarree())
ax3 = fig.add_subplot(gs[2, 1], projection = ccrs.PlateCarree())
ax4 = fig.add_subplot(gs[3, 0], projection = ccrs.PlateCarree())
ax5 = fig.add_subplot(gs[3, 1], projection = ccrs.PlateCarree())
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
plot_labels = ['b)','c)','d)','e)']
for ax, reg, num, lab in zip([ax2,ax3,ax4,ax5], reg_eu, [16,17,18,19], plot_labels):
    txx_reg = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    yr = weighted_area_lat(txx_reg).mean('lat').mean('lon').tasmax.idxmax('time').dt.year.values
    txx_up_to_year = txx_year_era_eur.tasmax.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(yr, 12, 31)))
    ranks = txx_up_to_year.rank(dim='time')
    rank_given_year = ranks.sel(time=str(yr))
    total_years = txx_up_to_year.sizes['time']
    
    # Apply categorization function
    categories = xr.where(rank_given_year.isnull(), np.nan, rank_given_year)
    categories = xr.apply_ufunc(categorize, rank_given_year, total_years,
                                vectorize=True, dask='parallelized', output_dtypes=[int])
    
    im = categories.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
    regionmask.defined_regions.ar6.land[num,num].plot(ax=ax, add_label=False, line_kws={'color': 'black', 'linewidth': 3})
    ax.set_title(f'$\mathbf{{{reg}}}$ ERA5 TXx record, $\mathbf{{{yr}}}$')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.coastlines()
    ax.set_extent([-10.5, 60.5, 29.5, 73], crs=ccrs.PlateCarree())
    ax.text(-0.03, 1.11, lab, transform=ax.transAxes, fontsize=20, va='top', ha='right')

category_labels = ['Coldest', '5% Coldest', '10% Coldest', '25% Coldest', 
                       'Near Normal', '25% Warmest', '10% Warmest', '5% Warmest', 'Warmest']
cbar = plt.colorbar(im, ax=[ax1,ax2,ax3,ax4,ax5], ticks=levels, fraction=0.05, shrink=0.5)
cbar.set_ticklabels(category_labels)
plt.savefig('figures/fig1.pdf', bbox_inches='tight', pad_inches=0)
plt.show()

## Time series (2)

In [ ]:
fig = plt.figure(figsize=(16, 20))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
eraref1960  = txx_year_era_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
rolmean = 1
rolling_mean = xr.concat([txx_year_hist_eur, txx_year_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))-ref30y_hist_ensmemb
mean = rolling_mean.mean('realiz')
std = rolling_mean.std('realiz')
min_val = rolling_mean.min('realiz')
max_val = rolling_mean.max('realiz')
ax1.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').tasmax, color='black', label='hist+ssp585')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(mean).mean('lat').mean('lon').tasmax-weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                 weighted_area_lat(mean).mean('lat').mean('lon').tasmax+weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                 color='black', alpha=0.2, label='', edgecolor='white')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(min_val).mean('lat').mean('lon').tasmax,
                 weighted_area_lat(max_val).mean('lat').mean('lon').tasmax,
                 color='black', alpha=0.25, label='', ec='white')

ax1.plot(txx_year_era_eur.rolling(time=rolmean, center=True).mean().time.dt.year,
         weighted_area_lat(txx_year_era_eur.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era, 
         color='royalblue', label='ERA5')

for i, year in enumerate(txx_year_stab_eur.exp.values):
    color = tYlOrRd(i / (len(txx_year_stab_eur.exp.values) - 1))   
    ax1.plot(txx_year_stab_eur.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
             weighted_area_lat(txx_year_stab_eur.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean, 
             color=color, label="NZ"+str(year))
ax1.set_ylim(-8,15)
ax1.set_xlim(1850, 3065)
ax1.set_xlabel("Time [year]")
ax1.set_ylabel("TXx anomaly [°C]")
ax1.set_title("EUROPE")
ax1.legend(ncol=2, fontsize=18, loc='lower center', shadow=True) # bbox_to_anchor=(1.17, -0.3)
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.text(-0.05, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    rolling_mean = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))
    mean = rolling_mean.mean('realiz')-ref30y_hist
    std = rolling_mean.std('realiz')
    min_val = rolling_mean.min('realiz')-ref30y_hist
    max_val = rolling_mean.max('realiz')-ref30y_hist
    ax.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').tasmax, color='black', label='hist+ssp585')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(mean).mean('lat').mean('lon').tasmax-weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                     weighted_area_lat(mean).mean('lat').mean('lon').tasmax+weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                     color='black', alpha=0.2, label='', edgecolor='white')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(min_val).mean('lat').mean('lon').tasmax,
                     weighted_area_lat(max_val).mean('lat').mean('lon').tasmax,
                     color='black', alpha=0.25, label='', ec='white')
    
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    ax.plot(txx_era.rolling(time=rolmean, center=True).mean().time.dt.year,
             weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era, 
             color='royalblue', label='ERA5')
    
    for i, year in enumerate(txx_year_stab_eur.exp.values):
        txx = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
        color = tYlOrRd(i / (len(txx_year_stab_eur.exp.values) - 1))   
        ax.plot(txx.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
                 weighted_area_lat(txx.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist, 
                 color=color, label="NZ"+str(year))
    ax.set_xlim(1850, 3065)
    ax.set_ylim(-8,15)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_title(f"{reg}")
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("TXx anomaly [°C]")
        ax.text(-0.1, 1.09, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.09, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("Time [year]")
        ax.set_ylabel("TXx anomaly [°C]")
        ax.text(-0.1, 1.09, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("Time [year]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.09, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/fig2.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## Rank histograms (3)

In [ ]:
rolmean = 1

fig = plt.figure(figsize=(16, 20))
gs = GridSpec(4, 2)
gs.update(wspace=0.1, hspace=0.4)
ax1 = fig.add_subplot(gs[0:2, 0:2])
histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
txx_year_histssp585 = xr.concat([txx_year_hist_eur, txx_year_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
txx_year_histssp585 = txx_year_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585 = weighted_area_lat(txx_year_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_year_era_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_year_era_eur.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era

ranks_era = np.zeros_like(txxanom_eurmean_era.values)
for i, txx_era in enumerate(txxanom_eurmean_era.values):
    combined = np.concatenate(([txx_era], txxanom_eurmean_histssp585[:, i]))
    ranked = stats.rankdata(combined, method='average')
    ranks_era[i] = ranked[0]

freq_table = np.zeros((40, 40)) # rows are experiments, columns are the ranks, values are the frequencies
rank_txx_realiz = txxanom_eurmean_histssp585.rank(dim='realiz')
for i, rea in enumerate(txxanom_eurmean_histssp585.realiz):
    freq, bin_edges = np.histogram(rank_txx_realiz.sel(realiz=rea), bins=np.arange(1, 42)-0.5)
    freq_table[i, :] = freq
    
freq_model12 = []
freq_model87 = []
for j, rea in enumerate(txxanom_eurmean_histssp585.realiz):
    freq_model12.append(np.quantile(freq_table[:, j], 0.125))
    freq_model87.append(np.quantile(freq_table[:, j], 0.875))

hist, bin_edges = np.histogram(ranks_era, bins=np.arange(1, 42)-0.5)
ax1.hist(ranks_era, bins=np.arange(42)-0.5, alpha=0.75, label='ERA5 histograms')

mean_freq = np.convolve(hist, np.ones(7)/7, mode='valid')
ax1.plot(np.arange(4, 38), mean_freq, alpha=0.85, color='navy', label='ERA5 7-bin window histogram slope', linewidth=4)

ax1.plot(np.arange(4, 38), np.convolve(freq_model12, np.ones(7)/7, mode='valid'), 
         linestyle='dashed',  color='black', linewidth=4)
ax1.plot(np.arange(4, 38), np.convolve(freq_model87, np.ones(7)/7, mode='valid'),
         linestyle='dashed',  color='black', linewidth=4, label='Perfect model 7-bin window histogram slope range (central p75)')
ax1.set_title('EUROPE')
ax1.set_xlabel('Rank')
ax1.set_ylabel('Frequency')
ax1.set_xlim(0,41)
ax1.legend(fontsize=18, loc='upper center', shadow=True)
ax1.text(-0.03, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    txx_year_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_year_histssp585 = txx_year_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_year_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    
    ranks_era = np.zeros_like(txxanom_eurmean_era.values)
    for i, txx_era in enumerate(txxanom_eurmean_era.values):
        combined = np.concatenate(([txx_era], txxanom_eurmean_histssp585[:, i]))
        ranked = stats.rankdata(combined, method='average')
        ranks_era[i] = ranked[0]
    freq_table = np.zeros((40, 40)) # rows are experiments, columns are the ranks, values are the frequencies
    rank_txx_realiz = txxanom_eurmean_histssp585.rank(dim='realiz')
    for i, rea in enumerate(txxanom_eurmean_histssp585.realiz):
        freq, bin_edges = np.histogram(rank_txx_realiz.sel(realiz=rea), bins=np.arange(1, 42)-0.5)
        freq_table[i, :] = freq
        
    freq_model12 = []
    freq_model87 = []
    for j, rea in enumerate(txxanom_eurmean_histssp585.realiz):
        freq_model12.append(np.quantile(freq_table[:, j], 0.125))
        freq_model87.append(np.quantile(freq_table[:, j], 0.875))
    hist, bin_edges = np.histogram(ranks_era, bins=np.arange(1, 42)-0.5)
    ax.hist(ranks_era, bins=np.arange(42)-0.5, alpha=0.75, label='ERA5 histograms')
    
    mean_freq = np.convolve(hist, np.ones(7)/7, mode='valid')
    ax.plot(np.arange(4, 38), mean_freq, alpha=0.85, color='navy', label='ERA5 7-bin window histogram slope', linewidth=4)
    
    ax.plot(np.arange(4, 38), np.convolve(freq_model12, np.ones(7)/7, mode='valid'), 
             linestyle='dashed',  color='black', linewidth=4)
    ax.plot(np.arange(4, 38), np.convolve(freq_model87, np.ones(7)/7, mode='valid'),
             linestyle='dashed',  color='black', linewidth=4, label='Perfect model 7-bin window histogram slope range (central p75)')
    ax.set_title(f'{reg}')
    ax.set_xlim(0,41)
    if reg == 'NEU':
        ax.set_ylabel("Frequency")
        ax.text(-0.04, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.text(-0.02, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("Rank")
        ax.set_ylabel("Frequency")
        ax.text(-0.04, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("Rank")       
        ax.text(-0.02, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/fig3.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## PDFs (4)

In [ ]:
rolmean=1

fig = plt.figure(figsize=(16, 17))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585_last30 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585_last30 = weighted_area_lat(txx_histssp585_last30).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
txx_histssp585_first50 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
txxanom_eurmean_histssp585_first50 = weighted_area_lat(txx_histssp585_first50).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean

sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_first50), ax=ax1, color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_last30), ax=ax1, color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)
ax1.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
ax1.set_title('EUROPE')
ax1.set_xlabel("TXx anomaly [°C]")
ax1.set_ylabel("Probability density")

for i, year in enumerate(txx_stab.exp.values):
    color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))   
    sns.kdeplot(txxanom_eurmean_stab.sel(exp=year), ax=ax1, color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
plt.legend(fontsize=20,loc='center right')
ax1.spines.right.set_visible(False)
ax1.spines.left.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.text(-0.04, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_histssp585_last30 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585_last30 = weighted_area_lat(txx_histssp585_last30).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    txx_histssp585_first50 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
    txxanom_eurmean_histssp585_first50 = weighted_area_lat(txx_histssp585_first50).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_first50), ax=ax, color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
    sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_last30), ax=ax, color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)    
    ax.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')    
    for i, year in enumerate(txx_stab.exp.values):
        color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))   
        sns.kdeplot(txxanom_eurmean_stab.sel(exp=year), ax=ax, color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
    ax.set_ylabel("")
    ax.set_title(f"{reg}")
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("Probability density")
        ax.text(-0.1, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.07, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("TXx anomaly [°C]")
        ax.set_ylabel("Probability density")
        ax.text(-0.1, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
        ax.set_yticks([0,0.1,0.2])
    if reg == 'MED':
        ax.set_xlabel("TXx anomaly [°C]")
        ax.spines.left.set_visible(False)
        ax.text(-0.07, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')

plt.savefig('figures/fig4.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## Risk ratios (5)

In [ ]:
rolmean=1
fig = plt.figure(figsize=(16, 18))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
ax1.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
for i, year in enumerate(txx_stab.exp.values):
    color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))  
    RR = []
    for thr in np.linspace(minvalue, maxvalue,60):
        count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
        count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
        p_0 = count_above/count_total
    
        count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
        count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
        p_1 = count_above/count_total
        if p_0 == 0:
            RR.append(np.nan)
        else:
            RR.append(p_1/p_0)
    ax1.plot(np.linspace(minvalue, maxvalue, 60), RR, color=color, linewidth=2, label="NZ"+str(year))
ax1.set_yscale('log')
ax1.set_ylabel("Risk ratios")
ax1.set_xlim(minvalue, maxvalue)
ax1.set_ylim(1,1000)
ax1.set_xlabel("TXx threshold [°C]")
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.legend(fontsize=20,loc='center right')
ax1.set_title('EUROPE')
ax1.text(-0.05, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
    maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
    for i, year in enumerate(txx_stab.exp.values):
        color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))  
        RR = []
        for thr in np.linspace(minvalue, maxvalue,60):
            count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
            count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
            p_0 = count_above/count_total
        
            count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
            count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
            p_1 = count_above/count_total
            if p_0 == 0:
                RR.append(np.nan)
            else:
                RR.append(p_1/p_0)
        ax.plot(np.linspace(minvalue, maxvalue, 60), RR, color=color, linewidth=2)
    ax.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3)
    ax.set_yscale('log')
    ax.set_ylim(1,1000)    
    ax.set_title(f"{reg}")
    ax.set_xlim(minvalue, maxvalue)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("Risk ratios")
        ax.text(-0.1, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("TXx threshold [°C]")
        ax.set_ylabel("Risk ratios")
        ax.text(-0.1, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("TXx threshold [°C]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/fig5.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## GWLs Europe (6)

In [ ]:
# GWL 1.5, 2, 3 °C +-0.2°C
# transient every member hist+ssp585 ensemble
# stab: early stable (100-450 years post emission), late stable (650-1000)

tas_yr_histssp = xr.concat([tas_yr_hist, tas_yr_ssp585], dim='time').rolling(time=10, center=True).mean()
tas_yr_histssp_mean = weighted_area_lat(tas_yr_histssp).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_histssp_mean_15 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 1.3) & (tas_yr_histssp_mean < 1.7), drop=True)
tas_yr_histssp_mean_20 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 1.8) & (tas_yr_histssp_mean < 2.2), drop=True)
tas_yr_histssp_mean_30 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 2.8) & (tas_yr_histssp_mean < 3.2), drop=True)


tas_yr_sta_early = tas_yr_sta.rolling(time=10, center=True).mean().sel(time=slice(cftime.DatetimeProlepticGregorian(100, 1, 1),cftime.DatetimeProlepticGregorian(450, 12, 31)))
tas_yr_sta_early_mean = weighted_area_lat(tas_yr_sta_early).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_sta_late = tas_yr_sta.rolling(time=10, center=True).mean().sel(time=slice(cftime.DatetimeProlepticGregorian(650, 1, 1),cftime.DatetimeProlepticGregorian(1000, 12, 31)))
tas_yr_sta_late_mean = weighted_area_lat(tas_yr_sta_late).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_sta_early_mean_15 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 1.3) & (tas_yr_sta_early_mean < 1.7), drop=True)
tas_yr_sta_early_mean_20 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 1.8) & (tas_yr_sta_early_mean < 2.2), drop=True)
tas_yr_sta_early_mean_30 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 2.8) & (tas_yr_sta_early_mean < 3.2), drop=True)

tas_yr_sta_late_mean_15 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 1.3) & (tas_yr_sta_late_mean < 1.7), drop=True)
tas_yr_sta_late_mean_20 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 1.8) & (tas_yr_sta_late_mean < 2.2), drop=True)
tas_yr_sta_late_mean_30 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 2.8) & (tas_yr_sta_late_mean < 3.2), drop=True)

In [ ]:
rolmean=1
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

# transient
txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
txxanom_eurmean_histssp585_15 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_15, drop=True)
txxanom_eurmean_histssp585_20 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_20, drop=True)
txxanom_eurmean_histssp585_30 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_30, drop=True)

# early,late stab
txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
txxanom_eurmean_stab_early_15 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_15, drop=True).sel(exp=2035)
txxanom_eurmean_stab_early_20 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_20, drop=True).sel(exp=2045)
txxanom_eurmean_stab_early_30 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_30, drop=True).sel(exp=2060)
txxanom_eurmean_stab_late_15 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_15, drop=True).sel(exp=2030)
txxanom_eurmean_stab_late_20 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_20, drop=True).sel(exp=2040)
txxanom_eurmean_stab_late_30 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_30, drop=True).sel(exp=2055)

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
plt.subplots_adjust(hspace=0.3)
sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_15), ax=axs[0], color='black', 
            fill=True, alpha=0.3, linewidth=2, label='transient')
sns.kdeplot(txxanom_eurmean_stab_early_15, ax=axs[0], color=tYlOrRd(1 / (len(tas_yr_sta.exp.values) - 1)), 
            fill=True, alpha=0.3, linewidth=2, label='early stable')
sns.kdeplot(txxanom_eurmean_stab_late_15, ax=axs[0], color=tYlOrRd(0 / (len(tas_yr_sta.exp.values) - 1)), 
            fill=True, alpha=0.3, linewidth=2, label='late stable')

axs[0].axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
axs[0].text(8,0.25,'1.5°C', fontweight='bold', fontsize=25, color=tYlOrRd(0.5 / (len(tas_yr_sta.exp.values) - 1)))
axs[0].set_ylabel("Probability density")
axs[0].set_xlim(-3,9)
axs[0].legend(fontsize=14, loc='center left')
axs[0].spines.right.set_visible(False)
axs[0].spines.top.set_visible(False)
axs[0].spines.bottom.set_visible(False)
axs[0].text(-0.1, 1.14, 'a)', transform=axs[0].transAxes, fontsize=20, va='top', ha='right')

sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_20), ax=axs[1], color='black', 
            fill=True, alpha=0.3, linewidth=2, label='transient')
sns.kdeplot(txxanom_eurmean_stab_early_20, ax=axs[1], color=tYlOrRd(3 / (len(tas_yr_sta.exp.values) - 1)),
            fill=True, alpha=0.3, linewidth=2, label='early stable')
sns.kdeplot(txxanom_eurmean_stab_late_20, ax=axs[1], color=tYlOrRd(2 / (len(tas_yr_sta.exp.values) - 1)), 
            fill=True, alpha=0.3, linewidth=2, label='late stable')

axs[1].axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
axs[1].text(8,0.25,'2°C', fontweight='bold', fontsize=25, color=tYlOrRd(2.5 / (len(tas_yr_sta.exp.values) - 1)))
axs[1].set_ylabel("Probability density")
axs[1].set_xlim(-3,9)
axs[1].legend(fontsize=14, loc='center left')
axs[1].spines.right.set_visible(False)
axs[1].spines.top.set_visible(False)
axs[1].spines.bottom.set_visible(False)
axs[1].text(-0.1, 1.14, 'b)', transform=axs[1].transAxes, fontsize=20, va='top', ha='right')

sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_30), ax=axs[2], color='black', 
            fill=True, alpha=0.3, linewidth=2, label='transient')
sns.kdeplot(txxanom_eurmean_stab_early_30, ax=axs[2], color=tYlOrRd(6 / (len(tas_yr_sta.exp.values) - 1)), 
            fill=True, alpha=0.3, linewidth=2, label='early stable')
sns.kdeplot(txxanom_eurmean_stab_late_30, ax=axs[2], color=tYlOrRd(5 / (len(tas_yr_sta.exp.values) - 1)),
            fill=True, alpha=0.3, linewidth=2, label='late stable')

axs[2].axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
axs[2].text(8,0.25,'3°C', fontweight='bold', fontsize=25, color=tYlOrRd(5.5 / (len(tas_yr_sta.exp.values) - 1)))
axs[2].set_ylabel("Probability density")
axs[2].set_xlim(-3,9)
axs[2].spines.right.set_visible(False)
axs[2].spines.top.set_visible(False)
axs[2].set_xlabel("TXx anomaly [°C]")
axs[2].legend(fontsize=14, loc='center left')
axs[2].spines.right.set_visible(False)
axs[2].spines.top.set_visible(False)
axs[2].text(-0.1, 1.14, 'c)', transform=axs[2].transAxes, fontsize=20, va='top', ha='right')

plt.savefig('figures/fig6.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

# Appendix figures

## ERA5 spatial monthly (A1)

In [ ]:
fig = plt.figure(figsize=(23, 24))
gs = GridSpec(5, 3)
gs.update(wspace=0.3, hspace=0.0)
levels = np.arange(1, 10)
cmap = plt.get_cmap('RdBu_r', len(levels)) 
cmap.set_bad(color='white')
norm = mcolors.BoundaryNorm(boundaries=np.arange(0.5, 9.5 + 1), ncolors=cmap.N)
sub_labels=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
label_cycle = itertools.cycle(sub_labels)
for j, month in enumerate([6, 7, 8]):
    txx = txx_mon_era_eur.where(txx_mon_era.time.dt.month==month, drop=True)
    ax1 = fig.add_subplot(gs[0, j], projection = ccrs.PlateCarree())
    yr = weighted_area_lat(txx).mean('lat').mean('lon').tasmax.idxmax('time').dt.year.values
    txx_up_to_year = txx.tasmax.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(yr, 12, 31)))
    ranks = txx_up_to_year.rank(dim='time')
    rank_given_year = ranks.sel(time=str(yr))
    total_years = txx_up_to_year.sizes['time']
    
    # Apply categorization function
    categories = xr.where(rank_given_year.isnull(), np.nan, rank_given_year)
    categories = xr.apply_ufunc(categorize, rank_given_year, total_years,
                                vectorize=True, dask='parallelized', output_dtypes=[int])
    
    im = categories.plot(ax=ax1, cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
    ax1.set_title(f'$\mathbf{{EUROPE}}$ ERA5 TXx record, $\mathbf{{0{month}.{yr}}}$')
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.coastlines()
    ax1.grid(False)
    ax1.text(-0.05, 1.11, f'{next(label_cycle)})', transform=ax1.transAxes, fontsize=20, va='top', ha='right')
    
    ax2 = fig.add_subplot(gs[1, j], projection = ccrs.PlateCarree())
    ax3 = fig.add_subplot(gs[2, j], projection = ccrs.PlateCarree())
    ax4 = fig.add_subplot(gs[3, j], projection = ccrs.PlateCarree())
    ax5 = fig.add_subplot(gs[4, j], projection = ccrs.PlateCarree())
    reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
    for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
        txx_reg = txx_mon_era.where(txx_mon_era.time.dt.month==month, drop=True).where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
        yr = weighted_area_lat(txx_reg).mean('lat').mean('lon').tasmax.idxmax('time').dt.year.values
        txx_up_to_year = txx.tasmax.sel(time=slice(cftime.DatetimeProlepticGregorian(1940, 1, 1),cftime.DatetimeProlepticGregorian(yr, 12, 31)))
        ranks = txx_up_to_year.rank(dim='time')
        rank_given_year = ranks.sel(time=str(yr))
        total_years = txx_up_to_year.sizes['time']
        
        # Apply categorization function
        categories = xr.where(rank_given_year.isnull(), np.nan, rank_given_year)
        categories = xr.apply_ufunc(categorize, rank_given_year, total_years,
                                    vectorize=True, dask='parallelized', output_dtypes=[int])
        
        im = categories.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
        ax.set_title(f'$\mathbf{{{reg}}}$ ERA5 TXx record, $\mathbf{{0{month}.{yr}}}$')
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.coastlines()
        ax.text(-0.03, 1.11, f"{next(label_cycle)})", transform=ax.transAxes, fontsize=20, va='top', ha='right')
    
category_labels = ['Coldest', '5% Coldest', '10% Coldest', '25% Coldest', 
                           'Near Normal', '25% Warmest', '10% Warmest', '5% Warmest', 'Warmest']
cbar = plt.colorbar(im, ax=fig.axes, ticks=levels, fraction=0.05, shrink=0.5)
cbar.set_ticklabels(category_labels)
plt.savefig('figures/figA1.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## Time series smooth (A2)

In [ ]:
fig = plt.figure(figsize=(16, 20))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
eraref1960  = txx_year_era_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
rolmean = 11
rolling_mean = xr.concat([txx_year_hist_eur, txx_year_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))-ref30y_hist_ensmemb
mean = rolling_mean.mean('realiz')
std = rolling_mean.std('realiz')
min_val = rolling_mean.min('realiz')
max_val = rolling_mean.max('realiz')
ax1.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').tasmax, color='black', label='hist+ssp585')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(mean).mean('lat').mean('lon').tasmax-weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                 weighted_area_lat(mean).mean('lat').mean('lon').tasmax+weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                 color='black', alpha=0.2, label='', edgecolor='white')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(min_val).mean('lat').mean('lon').tasmax,
                 weighted_area_lat(max_val).mean('lat').mean('lon').tasmax,
                 color='black', alpha=0.25, label='', ec='white')

ax1.plot(txx_year_era_eur.rolling(time=rolmean, center=True).mean().time.dt.year,
         weighted_area_lat(txx_year_era_eur.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era, 
         color='royalblue', label='ERA5')

for i, year in enumerate(txx_year_stab_eur.exp.values):
    color = tYlOrRd(i / (len(txx_year_stab_eur.exp.values) - 1))   
    ax1.plot(txx_year_stab_eur.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
             weighted_area_lat(txx_year_stab_eur.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean, 
             color=color, label="NZ"+str(year))
ax1.set_ylim(-5,12)
ax1.set_xlim(1850, 3065)
ax1.set_xlabel("Time [year]")
ax1.set_ylabel("11-year TXx anomaly [°C]")
ax1.set_title("EUROPE")
ax1.legend(ncol=2, fontsize=18, loc='lower center', shadow=True) # bbox_to_anchor=(1.17, -0.3)
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.text(-0.05, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    rolling_mean = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))
    mean = rolling_mean.mean('realiz')-ref30y_hist
    std = rolling_mean.std('realiz')
    min_val = rolling_mean.min('realiz')-ref30y_hist
    max_val = rolling_mean.max('realiz')-ref30y_hist
    ax.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').tasmax, color='black', label='hist+ssp585')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(mean).mean('lat').mean('lon').tasmax-weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                     weighted_area_lat(mean).mean('lat').mean('lon').tasmax+weighted_area_lat(std).mean('lat').mean('lon').tasmax,
                     color='black', alpha=0.2, label='', edgecolor='white')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(min_val).mean('lat').mean('lon').tasmax,
                     weighted_area_lat(max_val).mean('lat').mean('lon').tasmax,
                     color='black', alpha=0.25, label='', ec='white')
    
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    ax.plot(txx_era.rolling(time=rolmean, center=True).mean().time.dt.year,
             weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era, 
             color='royalblue', label='ERA5')
    
    for i, year in enumerate(txx_year_stab_eur.exp.values):
        txx = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
        color = tYlOrRd(i / (len(txx_year_stab_eur.exp.values) - 1))   
        ax.plot(txx.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
                 weighted_area_lat(txx.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist, 
                 color=color, label="NZ"+str(year))
    ax.set_xlim(1850, 3065)
    ax.set_ylim(-5,12)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_title(f"{reg}")
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("11-year TXx anomaly [°C]")
        ax.text(-0.1, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')

    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')

    if reg == 'EEU':
        ax.set_xlabel("Time [year]")
        ax.set_ylabel("11-year TXx anomaly [°C]")
        ax.text(-0.1, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')

    if reg == 'MED':
        ax.set_xlabel("Time [year]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/figA2.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## FAR (A3)

In [ ]:
rolmean=1
fig = plt.figure(figsize=(16, 18))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
ax1.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
for i, year in enumerate(txx_stab.exp.values):
    color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))  
    FAR = []
    for thr in np.linspace(minvalue, maxvalue,60):
        count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
        count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
        p_0 = count_above/count_total
    
        count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
        count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
        p_1 = count_above/count_total
        if p_1 == 0:
            FAR.append(np.nan)
        else:
            FAR.append(1-(p_0/p_1))
    ax1.plot(np.linspace(minvalue, maxvalue, 60), FAR, color=color, label="NZ"+str(year), linewidth=2)
ax1.set_ylabel("Fractions of attributable risk")
ax1.set_xlim(minvalue, maxvalue)
ax1.set_ylim(0,1)
ax1.set_xlabel("TXx threshold [°C]")
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.legend(fontsize=20,loc='center right')
ax1.set_title('EUROPE')
ax1.text(-0.07, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
    maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
    for i, year in enumerate(txx_stab.exp.values):
        color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))  
        FAR = []
        for thr in np.linspace(minvalue, maxvalue,60):
            count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
            count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
            p_0 = count_above/count_total
        
            count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
            count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
            p_1 = count_above/count_total
            if p_1 == 0:
                FAR.append(np.nan)
            else:
                FAR.append(1-(p_0/p_1))
        ax.plot(np.linspace(minvalue, maxvalue, 60), FAR, color=color, linewidth=2)
    ax.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3)
    ax.set_ylim(0,1)    
    ax.set_title(f"{reg}")
    ax.set_xlim(minvalue, maxvalue)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("Fractions of attributable risk")
        ax.text(-0.16, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("TXx threshold [°C]")
        ax.set_ylabel("Fractions of attributable risk")
        ax.text(-0.16, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("TXx threshold [°C]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/figA3.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## GWLs European regions (A4)

In [ ]:
# GWL 1.5, 2, 3 °C +-0.2°C
# transient every member hist+ssp585 ensemble
# stab: early stable (100-450 years post emission), late stable (650-1000)

tas_yr_histssp = xr.concat([tas_yr_hist, tas_yr_ssp585], dim='time').rolling(time=10, center=True).mean()
tas_yr_histssp_mean = weighted_area_lat(tas_yr_histssp).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_histssp_mean_15 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 1.3) & (tas_yr_histssp_mean < 1.7), drop=True)
tas_yr_histssp_mean_20 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 1.8) & (tas_yr_histssp_mean < 2.2), drop=True)
tas_yr_histssp_mean_30 = tas_yr_histssp_mean.where((tas_yr_histssp_mean > 2.8) & (tas_yr_histssp_mean < 3.2), drop=True)


tas_yr_sta_early = tas_yr_sta.rolling(time=10, center=True).mean().sel(time=slice(cftime.DatetimeProlepticGregorian(100, 1, 1),cftime.DatetimeProlepticGregorian(450, 12, 31)))
tas_yr_sta_early_mean = weighted_area_lat(tas_yr_sta_early).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_sta_late = tas_yr_sta.rolling(time=10, center=True).mean().sel(time=slice(cftime.DatetimeProlepticGregorian(650, 1, 1),cftime.DatetimeProlepticGregorian(1000, 12, 31)))
tas_yr_sta_late_mean = weighted_area_lat(tas_yr_sta_late).mean('lat').mean('lon')-tasref_50yrhist_yr

tas_yr_sta_early_mean_15 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 1.3) & (tas_yr_sta_early_mean < 1.7), drop=True)
tas_yr_sta_early_mean_20 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 1.8) & (tas_yr_sta_early_mean < 2.2), drop=True)
tas_yr_sta_early_mean_30 = tas_yr_sta_early_mean.where((tas_yr_sta_early_mean > 2.8) & (tas_yr_sta_early_mean < 3.2), drop=True)

tas_yr_sta_late_mean_15 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 1.3) & (tas_yr_sta_late_mean < 1.7), drop=True)
tas_yr_sta_late_mean_20 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 1.8) & (tas_yr_sta_late_mean < 2.2), drop=True)
tas_yr_sta_late_mean_30 = tas_yr_sta_late_mean.where((tas_yr_sta_late_mean > 2.8) & (tas_yr_sta_late_mean < 3.2), drop=True)

In [ ]:
rolmean=1

fig = plt.figure(figsize=(28, 15))
gs = GridSpec(3, 4)
gs.update(wspace=0.3, hspace=0.3)
sub_labels=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
label_cycle = itertools.cycle(sub_labels)
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for j, reg in enumerate(reg_eu):

    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    # transient
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    txxanom_eurmean_histssp585_15 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_15, drop=True)
    txxanom_eurmean_histssp585_20 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_20, drop=True)
    txxanom_eurmean_histssp585_30 = txxanom_eurmean_histssp585.where(tas_yr_histssp_mean_30, drop=True)
    
    # early,late stab
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    txxanom_eurmean_stab_early_15 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_15, drop=True).sel(exp=2035)
    txxanom_eurmean_stab_early_20 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_20, drop=True).sel(exp=2045)
    txxanom_eurmean_stab_early_30 = txxanom_eurmean_stab.where(tas_yr_sta_early_mean_30, drop=True).sel(exp=2060)
    txxanom_eurmean_stab_late_15 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_15, drop=True).sel(exp=2030)
    txxanom_eurmean_stab_late_20 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_20, drop=True).sel(exp=2040)
    txxanom_eurmean_stab_late_30 = txxanom_eurmean_stab.where(tas_yr_sta_late_mean_30, drop=True).sel(exp=2055)

    ax1 = fig.add_subplot(gs[0, j])
    ax2 = fig.add_subplot(gs[1, j])
    ax3 = fig.add_subplot(gs[2, j])
    sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_15), ax=ax1, color='black', 
                fill=True, alpha=0.3, linewidth=2, label='transient')
    sns.kdeplot(txxanom_eurmean_stab_early_15, ax=ax1, color=tYlOrRd(1 / (len(tas_yr_sta.exp.values) - 1)), 
                fill=True, alpha=0.3, linewidth=2, label='early stable')
    sns.kdeplot(txxanom_eurmean_stab_late_15, ax=ax1, color=tYlOrRd(0 / (len(tas_yr_sta.exp.values) - 1)), 
                fill=True, alpha=0.3, linewidth=2, label='late stable')
    
    ax1.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
    ax1.set_xlim(-3,9)
    ax1.spines.right.set_visible(False)
    ax1.spines.top.set_visible(False)
    ax1.spines.bottom.set_visible(False)
    ax1.spines.left.set_visible(False)
    ax1.set_ylabel("")
    ax1.text(-0.1, 1.14, f"{next(label_cycle)})", transform=ax1.transAxes, fontsize=20, va='top', ha='right')
    ax1.set_title(f'$\mathbf{{{reg}}}$')

    sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_20), ax=ax2, color='black', 
                fill=True, alpha=0.3, linewidth=2, label='transient')
    sns.kdeplot(txxanom_eurmean_stab_early_20, ax=ax2, color=tYlOrRd(3 / (len(tas_yr_sta.exp.values) - 1)),
                fill=True, alpha=0.3, linewidth=2, label='early stable')
    sns.kdeplot(txxanom_eurmean_stab_late_20, ax=ax2, color=tYlOrRd(2 / (len(tas_yr_sta.exp.values) - 1)), 
                fill=True, alpha=0.3, linewidth=2, label='late stable')
    
    ax2.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
    ax2.set_xlim(-3,10)
    ax2.spines.right.set_visible(False)
    ax2.spines.top.set_visible(False)
    ax2.spines.bottom.set_visible(False)
    ax2.spines.left.set_visible(False)
    ax2.set_ylabel("")
    ax2.text(-0.1, 1.14, f"{next(label_cycle)})", transform=ax2.transAxes, fontsize=20, va='top', ha='right')
    
    sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_30), ax=ax3, color='black', 
                fill=True, alpha=0.3, linewidth=2, label='transient')
    sns.kdeplot(txxanom_eurmean_stab_early_30, ax=ax3, color=tYlOrRd(6 / (len(tas_yr_sta.exp.values) - 1)), 
                fill=True, alpha=0.3, linewidth=2, label='early stable')
    sns.kdeplot(txxanom_eurmean_stab_late_30, ax=ax3, color=tYlOrRd(5 / (len(tas_yr_sta.exp.values) - 1)),
                fill=True, alpha=0.3, linewidth=2, label='late stable')
    
    ax3.axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
    ax3.set_xlim(-3,12)
    ax3.spines.right.set_visible(False)
    ax3.spines.top.set_visible(False)
    ax3.set_xlabel("TXx anomaly [°C]")
    ax3.set_ylabel("")
    ax3.spines.right.set_visible(False)
    ax3.spines.top.set_visible(False)
    ax3.spines.left.set_visible(False)
    ax3.text(-0.1, 1.14, f"{next(label_cycle)})", transform=ax3.transAxes, fontsize=20, va='top', ha='right')

    if j == 0:
        ax1.spines.left.set_visible(True)
        ax2.spines.left.set_visible(True)
        ax3.spines.left.set_visible(True)
        ax1.set_ylabel("Probability density")
        ax2.set_ylabel("Probability density")
        ax3.set_ylabel("Probability density")
        ax1.text(-8.7,0.11,'1.5°C', fontweight='bold', fontsize=30, color=tYlOrRd(0.5 / (len(tas_yr_sta.exp.values) - 1)))  
        ax2.text(-8.7,0.11,'2°C', fontweight='bold', fontsize=30, color=tYlOrRd(2.5 / (len(tas_yr_sta.exp.values) - 1)))
        ax3.text(-8.7,0.11,'3°C', fontweight='bold', fontsize=30, color=tYlOrRd(5.5 / (len(tas_yr_sta.exp.values) - 1)))
        
    if j == 3:
        ax1.legend(fontsize=16, loc='center right', bbox_to_anchor=(1.3, 0.5))
        ax2.legend(fontsize=16, loc='center right', bbox_to_anchor=(1.3, 0.5))
        ax3.legend(fontsize=16, loc='center right', bbox_to_anchor=(1.3, 0.5))
        
plt.savefig('figures/figA4.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## first vs last 500 yrs (A5)

In [ ]:
rolmean=1
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585_last30 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585_last30 = weighted_area_lat(txx_histssp585_last30).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
txx_histssp585_first50 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
txxanom_eurmean_histssp585_first50 = weighted_area_lat(txx_histssp585_first50).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
plt.subplots_adjust(hspace=0.3)
txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
txxanom_eurmean_stab = txxanom_eurmean_stab.sel(time=slice(cftime.DatetimeProlepticGregorian(1, 1, 1),cftime.DatetimeProlepticGregorian(500, 12, 31)))

sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_first50), ax=axs[0], color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_last30), ax=axs[0], color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)
axs[0].axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
axs[0].set_title('EUROPE, years 1-500')
axs[0].set_ylabel("Probability density")
axs[0].set_xlim(-3,9)
for i, year in enumerate(txx_stab.exp.values):
    color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))   
    sns.kdeplot(txxanom_eurmean_stab.sel(exp=year), ax=axs[0], color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
axs[0].spines.right.set_visible(False)
axs[0].spines.top.set_visible(False)
axs[0].spines.bottom.set_visible(False)

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
txxanom_eurmean_stab = txxanom_eurmean_stab.sel(time=slice(cftime.DatetimeProlepticGregorian(501, 1, 1),cftime.DatetimeProlepticGregorian(1000, 12, 31)))

sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_first50), ax=axs[1], color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
sns.kdeplot(concat_ensmem(txxanom_eurmean_histssp585_last30), ax=axs[1], color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)
axs[1].axvline(txxanom_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
axs[1].set_title('EUROPE, years 501-1000')
axs[1].set_xlim(-3,9)
axs[1].set_xlabel("TXx anomaly [°C]")
axs[1].set_ylabel("Probability density")

for i, year in enumerate(txx_stab.exp.values):
    color = tYlOrRd(i / (len(txx_stab.exp.values) - 1))   
    sns.kdeplot(txxanom_eurmean_stab.sel(exp=year), ax=axs[1], color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
axs[1].legend(fontsize=20, loc='center right', bbox_to_anchor=(1.35, 1.2))
axs[1].spines.right.set_visible(False)
axs[1].spines.top.set_visible(False)
plt.savefig('figures/figA5.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## TX90p time series (A6)

In [ ]:
fig = plt.figure(figsize=(16, 20))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])

rolmean = 1
rolling_mean = xr.concat([tx90p_yr_hist_eur, tx90p_yr_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))
mean = rolling_mean.mean('realiz')
std = rolling_mean.std('realiz')
min_val = rolling_mean.min('realiz')
max_val = rolling_mean.max('realiz')
ax1.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').TX90p, color='black', label='hist+ssp585')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(mean).mean('lat').mean('lon').TX90p-weighted_area_lat(std).mean('lat').mean('lon').TX90p,
                 weighted_area_lat(mean).mean('lat').mean('lon').TX90p+weighted_area_lat(std).mean('lat').mean('lon').TX90p,
                 color='black', alpha=0.2, label='', edgecolor='white')
ax1.fill_between(mean.time.dt.year,
                 weighted_area_lat(min_val).mean('lat').mean('lon').TX90p,
                 weighted_area_lat(max_val).mean('lat').mean('lon').TX90p,
                 color='black', alpha=0.25, label='', ec='white')

ax1.plot(tx90p_yr_era5_eur.rolling(time=rolmean, center=True).mean().time.dt.year,
         weighted_area_lat(tx90p_yr_era5_eur.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p, 
         color='royalblue', label='ERA5')

for i, year in enumerate(tx90p_yr_sta_eur.exp.values):
    color = tYlOrRd(i / (len(tx90p_yr_sta_eur.exp.values) - 1))   
    plt.plot(tx90p_yr_sta_eur.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
             weighted_area_lat(tx90p_yr_sta_eur.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p, 
             color=color, label="NZ"+str(year))

ax1.set_ylim(0,365)
ax1.set_xlim(1850, 3065)
ax1.set_xlabel("Time [year]")
ax1.set_ylabel("TX90p [days/year]")
ax1.set_title("EUROPE")
ax1.legend(ncol=2, fontsize=18, loc='upper center', shadow=True) # bbox_to_anchor=(1.17, -0.3)
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.text(-0.05, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    tx90p_hist = tx90p_yr_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    tx90p_ssp585 = tx90p_yr_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    rolling_mean = xr.concat([tx90p_hist, tx90p_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    rolling_mean = rolling_mean.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(2100, 12, 31)))
    mean = rolling_mean.mean('realiz')
    std = rolling_mean.std('realiz')
    min_val = rolling_mean.min('realiz')
    max_val = rolling_mean.max('realiz')
    ax.plot(mean.time.dt.year, weighted_area_lat(mean).mean('lat').mean('lon').TX90p, color='black', label='hist+ssp585')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(mean).mean('lat').mean('lon').TX90p-weighted_area_lat(std).mean('lat').mean('lon').TX90p,
                     weighted_area_lat(mean).mean('lat').mean('lon').TX90p+weighted_area_lat(std).mean('lat').mean('lon').TX90p,
                     color='black', alpha=0.2, label='', edgecolor='white')
    ax.fill_between(mean.time.dt.year,
                     weighted_area_lat(min_val).mean('lat').mean('lon').TX90p,
                     weighted_area_lat(max_val).mean('lat').mean('lon').TX90p,
                     color='black', alpha=0.25, label='', ec='white')
    
    tx90p_era = tx90p_yr_era5.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    ax.plot(tx90p_era.rolling(time=rolmean, center=True).mean().time.dt.year,
             weighted_area_lat(tx90p_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p, 
             color='royalblue', label='ERA5')
    
    for i, year in enumerate(tx90p_yr_sta_eur.exp.values):
        tx90p = tx90p_yr_sta.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
        color = tYlOrRd(i / (len(tx90p_yr_sta_eur.exp.values) - 1))   
        ax.plot(tx90p.sel(exp=year).rolling(time=rolmean, center=True).mean().time.dt.year + 2030+5*i, 
                 weighted_area_lat(tx90p.sel(exp=year).rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p, 
                 color=color, label="NZ"+str(year))
    ax.set_xlim(1850, 3065)
    ax.set_ylim(0,365)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_title(f"{reg}")
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("TX90p [days/year]")
        ax.text(-0.1, 1.09, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.09, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("Time [year]")
        ax.set_ylabel("TX90p [days/year]")
        ax.text(-0.1, 1.09, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("Time [year]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.09, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/figA6.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## TX90p PDFs (A7)

In [ ]:
rolmean=1

fig = plt.figure(figsize=(16, 17))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
tx90p_hist = tx90p_yr_hist_eur
tx90p_ssp585 = tx90p_yr_ssp585_eur
tx90p_era = tx90p_yr_era5_eur
tx90p_stab = tx90p_yr_sta_eur

tx90p_histssp585 = xr.concat([tx90p_hist, tx90p_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
tx90p_histssp585_last30 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
tx90p_eurmean_histssp585_last30 = weighted_area_lat(tx90p_histssp585_last30).mean('lat').mean('lon').TX90p
tx90p_histssp585_first50 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
tx90p_eurmean_histssp585_first50 = weighted_area_lat(tx90p_histssp585_first50).mean('lat').mean('lon').TX90p 

tx90p_eurmean_era = weighted_area_lat(tx90p_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
tx90p_eurmean_eramax = tx90p_eurmean_era.sel(time=tx90p_eurmean_era.idxmax('time'))

tx90p_eurmean_stab = weighted_area_lat(tx90p_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p

sns.kdeplot(concat_ensmem(tx90p_eurmean_histssp585_first50), ax=ax1, color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
sns.kdeplot(concat_ensmem(tx90p_eurmean_histssp585_last30), ax=ax1, color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)
ax1.axvline(tx90p_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
ax1.set_title('EUROPE')
ax1.set_xlabel("TX90p [days/year]")
ax1.set_ylabel("Probability density")
ax1.set_xlim(0,250)

for i, year in enumerate(tx90p_stab.exp.values):
    color = tYlOrRd(i / (len(tx90p_stab.exp.values) - 1))   
    sns.kdeplot(tx90p_eurmean_stab.sel(exp=year), ax=ax1, color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
plt.legend(fontsize=20,loc='upper right')
ax1.spines.right.set_visible(False)
ax1.spines.left.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.text(-0.04, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    tx90p_hist = tx90p_yr_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    tx90p_ssp585 = tx90p_yr_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    tx90p_era = tx90p_yr_era5.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    tx90p_stab = tx90p_yr_sta.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    tx90p_histssp585 = xr.concat([tx90p_hist, tx90p_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    tx90p_histssp585_last30 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    tx90p_eurmean_histssp585_last30 = weighted_area_lat(tx90p_histssp585_last30).mean('lat').mean('lon').TX90p 
    tx90p_histssp585_first50 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1850, 1, 1),cftime.DatetimeProlepticGregorian(1899, 12, 31)))
    tx90p_eurmean_histssp585_first50 = weighted_area_lat(tx90p_histssp585_first50).mean('lat').mean('lon').TX90p

    tx90p_eurmean_era = weighted_area_lat(tx90p_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
    tx90p_eurmean_eramax = tx90p_eurmean_era.sel(time=tx90p_eurmean_era.idxmax('time'))
    
    tx90p_eurmean_stab = weighted_area_lat(tx90p_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
    sns.kdeplot(concat_ensmem(tx90p_eurmean_histssp585_first50), ax=ax, color='grey', label='1850-1899', fill=True, alpha=0.3, linewidth=2)
    sns.kdeplot(concat_ensmem(tx90p_eurmean_histssp585_last30), ax=ax, color='black', label='1995-2024', fill=True, alpha=0.3, linewidth=2)    
    ax.axvline(tx90p_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')    
    for i, year in enumerate(tx90p_stab.exp.values):
        color = tYlOrRd(i / (len(tx90p_stab.exp.values) - 1))   
        sns.kdeplot(tx90p_eurmean_stab.sel(exp=year), ax=ax, color=color, label="NZ"+str(year), fill=True, alpha=0.3, linewidth=2)
    ax.set_ylabel("")
    ax.set_title(f"{reg}")
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_xlim(0,250)
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("Probability density")
        ax.text(-0.1, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.07, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("TX90p [days/year]")
        ax.set_ylabel("Probability density")
        ax.text(-0.1, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
        #ax.set_yticks([0,0.1,0.2])
    if reg == 'MED':
        ax.set_xlabel("TX90p [days/year]")
        ax.spines.left.set_visible(False)
        ax.text(-0.07, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/figA7.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

## TX90p risk ratios (A8)

In [ ]:
rolmean=1
fig = plt.figure(figsize=(16, 18))
gs = GridSpec(4, 2)
gs.update(wspace=0.2, hspace=0.5)
ax1 = fig.add_subplot(gs[0:2, 0:2])
tx90p_hist = tx90p_yr_hist_eur
tx90p_ssp585 = tx90p_yr_ssp585_eur
tx90p_era = tx90p_yr_era5_eur
tx90p_stab = tx90p_yr_sta_eur

tx90p_histssp585 = xr.concat([tx90p_hist, tx90p_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
tx90p_histssp585 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
tx90p_eurmean_histssp585 = weighted_area_lat(tx90p_histssp585).mean('lat').mean('lon').TX90p

tx90p_eurmean_era = weighted_area_lat(tx90p_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
tx90p_eurmean_eramax = tx90p_eurmean_era.sel(time=tx90p_eurmean_era.idxmax('time'))

tx90p_eurmean_stab = weighted_area_lat(tx90p_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
minvalue = tx90p_eurmean_stab.sel(exp=2030).sel(time=tx90p_eurmean_stab.sel(exp=2030).idxmin('time'))
maxvalue = tx90p_eurmean_stab.sel(exp=2060).sel(time=tx90p_eurmean_stab.sel(exp=2060).idxmax('time'))
ax1.axvline(tx90p_eurmean_eramax, color='royalblue', linewidth=3, label='ERA5 record')
for i, year in enumerate(tx90p_stab.exp.values):
    color = tYlOrRd(i / (len(tx90p_stab.exp.values) - 1))  
    RR = []
    for thr in np.linspace(minvalue, maxvalue, 60):
        count_above = (tx90p_eurmean_histssp585 > thr).sum().item()
        count_total = (tx90p_eurmean_histssp585 >= tx90p_eurmean_histssp585.min().item()).sum().item()
        p_0 = count_above/count_total
    
        count_above = (tx90p_eurmean_stab.sel(exp=year) > thr).sum().item()
        count_total = (tx90p_eurmean_stab.sel(exp=year) >= tx90p_eurmean_stab.sel(exp=year).min().item()).sum().item()
        p_1 = count_above/count_total
        if p_0 == 0:
            RR.append(np.nan)
        else:
            RR.append(p_1/p_0)
    ax1.plot(np.linspace(minvalue, maxvalue, 60), RR, color=color, linewidth=2, label="NZ"+str(year))
ax1.set_yscale('log')
ax1.set_ylabel("Risk ratios")
ax1.set_xlim(minvalue, 180)
ax1.set_ylim(1,2000)
ax1.set_xlabel("TX90p threshold [days/year]")
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.legend(fontsize=20,loc='center right')
ax1.set_title('EUROPE')
ax1.text(-0.05, 1.05, 'a)', transform=ax1.transAxes, fontsize=20, va='top', ha='right')

ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for ax, reg in zip([ax2,ax3,ax4,ax5], reg_eu):
    tx90p_hist = tx90p_yr_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    tx90p_ssp585 = tx90p_yr_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    tx90p_era = tx90p_yr_era5.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    tx90p_stab = tx90p_yr_sta.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
       
    tx90p_histssp585 = xr.concat([tx90p_hist, tx90p_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    tx90p_histssp585 = tx90p_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    tx90p_eurmean_histssp585 = weighted_area_lat(tx90p_histssp585).mean('lat').mean('lon').TX90p 
    
    tx90p_eurmean_era = weighted_area_lat(tx90p_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
    tx90p_eurmean_eramax = tx90p_eurmean_era.sel(time=tx90p_eurmean_era.idxmax('time'))
    
    tx90p_eurmean_stab = weighted_area_lat(tx90p_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').TX90p
    minvalue = tx90p_eurmean_stab.sel(exp=2030).sel(time=tx90p_eurmean_stab.sel(exp=2030).idxmin('time'))
    maxvalue = tx90p_eurmean_stab.sel(exp=2060).sel(time=tx90p_eurmean_stab.sel(exp=2060).idxmax('time'))
    for i, year in enumerate(tx90p_stab.exp.values):
        color = tYlOrRd(i / (len(tx90p_stab.exp.values) - 1))  
        RR = []
        for thr in np.linspace(minvalue, maxvalue, 60):
            count_above = (tx90p_eurmean_histssp585 > thr).sum().item()
            count_total = (tx90p_eurmean_histssp585 >= tx90p_eurmean_histssp585.min().item()).sum().item()
            p_0 = count_above/count_total
        
            count_above = (tx90p_eurmean_stab.sel(exp=year) > thr).sum().item()
            count_total = (tx90p_eurmean_stab.sel(exp=year) >= tx90p_eurmean_stab.sel(exp=year).min().item()).sum().item()
            p_1 = count_above/count_total
            if p_0 == 0:
                RR.append(np.nan)
            else:
                RR.append(p_1/p_0)
        ax.plot(np.linspace(minvalue, maxvalue, 60), RR, color=color, linewidth=2)
    ax.axvline(tx90p_eurmean_eramax, color='royalblue', linewidth=3)
    ax.set_yscale('log')
    ax.set_ylim(1,2000)    
    ax.set_title(f"{reg}")
    ax.set_xlim(minvalue, 180)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    if reg == 'NEU':
        ax.spines.bottom.set_visible(False)
        ax.set_ylabel("Risk ratios")
        ax.text(-0.1, 1.14, 'b)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'WCE':
        ax.spines.bottom.set_visible(False)
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'c)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'EEU':
        ax.set_xlabel("TX90p threshold [days/year]")
        ax.set_ylabel("Risk ratios")
        ax.text(-0.1, 1.14, 'd)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
    if reg == 'MED':
        ax.set_xlabel("TX90p threshold [days/year]")
        ax.spines.left.set_visible(False)
        ax.text(-0.1, 1.14, 'e)', transform=ax.transAxes, fontsize=20, va='top', ha='right')
plt.savefig('figures/figA8.pdf', transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

# Appendix tables

## Time series stats (1)

ERA5 (1995-2024)

In [ ]:
rolmean = 1

eraref1960  = txx_year_era_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
eraref1960_spamean = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
era1995_2024  = txx_year_era_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
era1995_2024_spamean= weighted_area_lat(era1995_2024).mean('lat').mean('lon').tasmax
slope, intercept, r_value, p_value, std_err = stats.linregress((era1995_2024_spamean-eraref1960_spamean).time.dt.year.values, (era1995_2024_spamean-eraref1960_spamean).values)
print(f"\033[1mEUROPE\033[0m")
print(f"ERA5 1995-2024 mean: {np.round((era1995_2024_spamean-eraref1960_spamean).mean('time').values, 1)} °C")
print(f"ERA5 1995-2024 std: {np.round((era1995_2024_spamean-eraref1960_spamean).std('time').values, 1)} °C")
print(f"ERA5 1995-2024 trend: {np.round(slope, 3)} °C/year")
print(f"ERA5 1995-2024 R-squared: {np.round(r_value**2, 2)} ")
print(f"ERA5 1995-2024 p-value: {np.round(p_value, 6)}")
print("")
reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for reg in reg_eu:
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    eraref1960_spamean = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    era1995_2024  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    era1995_2024_spamean= weighted_area_lat(era1995_2024).mean('lat').mean('lon').tasmax
    slope, intercept, r_value, p_value, std_err = stats.linregress((era1995_2024_spamean-eraref1960_spamean).time.dt.year.values, (era1995_2024_spamean-eraref1960_spamean).values)
    print(f"\033[1m{reg}\033[0m")
    print(f"ERA5 1995-2024 mean: {np.round((era1995_2024_spamean-eraref1960_spamean).mean('time').values, 1)} °C")
    print(f"ERA5 1995-2024 std: {np.round((era1995_2024_spamean-eraref1960_spamean).std('time').values, 1)} °C")
    print(f"ERA5 1995-2024 trend: {np.round(slope, 3)} °C/year")
    print(f"ERA5 1995-2024 R-squared: {np.round(r_value**2, 2)} ")
    print(f"ERA5 1995-2024 p-value: {np.round(p_value, 6)}")
    print("")

hist+ssp585 (1995-2024)

In [ ]:
# ENSEMBLE RANGE
rolmean = 1

histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
histref1960_spamean_memb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

hist_ssp = xr.concat([txx_year_hist_eur, txx_year_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
hist_ssp1995_2024 = hist_ssp.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
hist_ssp1995_2024_spamean_memb = weighted_area_lat(hist_ssp1995_2024).mean('lat').mean('lon').tasmax
mean, std, trend = [],[],[]
for rea in hist_ssp.realiz: 
    mean.append((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-histref1960_spamean_memb.sel(realiz=rea)).mean('time'))
    std.append((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-histref1960_spamean_memb.sel(realiz=rea)).std('time'))
    slope, intercept, r_value, p_value, std_err = stats.linregress((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-histref1960_spamean_memb.sel(realiz=rea)).time.dt.year.values, 
                                                                   (hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-histref1960_spamean_memb.sel(realiz=rea)).values)
    trend.append(slope)
print(f"\033[1mEUROPE\033[0m")
print(f"hist+ssp585 1995-2024 mean: ({np.round(np.min(mean), 1)}, {np.round(np.max(mean), 1)}) °C")
print(f"hist+ssp585 1995-2024 std: ({np.round(np.min(std), 1)}, {np.round(np.max(std), 1)}) °C")
print(f"hist+ssp585 1995-2024 trend: ({np.round(np.min(trend), 3)}, {np.round(np.max(trend), 3)}) °C/year")
print("")


reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for reg in reg_eu:
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    hist_ssp = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    hist_ssp1995_2024 = hist_ssp.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    hist_ssp1995_2024_spamean_memb = weighted_area_lat(hist_ssp1995_2024).mean('lat').mean('lon').tasmax
    mean, std, trend = [],[],[]
    for rea in hist_ssp.realiz: 
        mean.append((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-ref30y_hist.sel(realiz=rea)).mean('time'))
        std.append((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-ref30y_hist.sel(realiz=rea)).std('time'))
        slope, intercept, r_value, p_value, std_err = stats.linregress((hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-ref30y_hist.sel(realiz=rea)).time.dt.year.values, 
                                                                       (hist_ssp1995_2024_spamean_memb.sel(realiz=rea)-ref30y_hist.sel(realiz=rea)).values)
        trend.append(slope)
    print(f"\033[1m{reg}\033[0m")
    print(f"hist+ssp585 1995-2024 mean: ({np.round(np.min(mean), 1)}, {np.round(np.max(mean), 1)}) °C")
    print(f"hist+ssp585 1995-2024 std: ({np.round(np.min(std), 1)}, {np.round(np.max(std), 1)}) °C")
    print(f"hist+ssp585 1995-2024 trend: ({np.round(np.min(trend), 3)}, {np.round(np.max(trend), 3)}) °C/year")
    print("")

In [ ]:
# ENSEMBLE MEAN
rolmean = 1

histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
histref1960_spamean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
histref1960_spamean_memb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

hist_ssp = xr.concat([txx_year_hist_eur, txx_year_ssp585_eur], dim='time').rolling(time=rolmean, center=True).mean()
hist_ssp1995_2024 = hist_ssp.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
hist_ssp1995_2024_spamean_ensmean = weighted_area_lat(hist_ssp1995_2024).mean('lat').mean('lon').mean('realiz').tasmax
slope, intercept, r_value, p_value, std_err = stats.linregress((hist_ssp1995_2024_spamean_ensmean-histref1960_spamean).time.dt.year.values, 
                                                               (hist_ssp1995_2024_spamean_ensmean-histref1960_spamean).values)

print(f"\033[1mEUROPE\033[0m")
print(f"hist+ssp585 1995-2024 mean: {np.round((hist_ssp1995_2024_spamean_ensmean-histref1960_spamean).mean('time').values, 1)} °C")
print(f"hist+ssp585 1995-2024 std: {np.round((hist_ssp1995_2024_spamean_ensmean-histref1960_spamean).std('time').values, 1)} °C")
print(f"hist+ssp585 1995-2024 trend: {np.round(slope, 3)} °C/year")
print(f"hist+ssp585 1995-2024 R-squared: {np.round(r_value**2, 2)} ")
print(f"hist+ssp585 1995-2024 p-value: {np.round(p_value, 6)}")
print("")

reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for reg in reg_eu:
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    hist_ssp = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    hist_ssp1995_2024 = hist_ssp.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    hist_ssp1995_2024_spamean_ensmean = weighted_area_lat(hist_ssp1995_2024).mean('lat').mean('lon').mean('realiz').tasmax
    slope, intercept, r_value, p_value, std_err = stats.linregress((hist_ssp1995_2024_spamean_ensmean-ref30y_hist).time.dt.year.values, 
                                                                   (hist_ssp1995_2024_spamean_ensmean-ref30y_hist).values)
    
    print(f"\033[1m{reg}\033[0m")
    print(f"ERA5 1995-2024 mean: {np.round((hist_ssp1995_2024_spamean_ensmean-ref30y_hist).mean('time').values, 1)} °C")
    print(f"ERA5 1995-2024 std: {np.round((hist_ssp1995_2024_spamean_ensmean-ref30y_hist).std('time').values, 1)} °C")
    print(f"ERA5 1995-2024 trend: {np.round(slope, 3)} °C/year")
    print(f"ERA5 1995-2024 R-squared: {np.round(r_value**2, 2)} ")
    print(f"ERA5 1995-2024 p-value: {np.round(p_value, 6)}")
    print("")

NZ experiments

In [ ]:
rolmean = 1

histref1960 = txx_year_hist_eur.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref1960_mean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref1960_memb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

for i, year in enumerate(txx_year_stab_eur.exp.values):
    print(f"\033[1mNZ{year}\033[0m")
    txx_stab = txx_year_stab_eur.sel(exp=year).rolling(time=rolmean, center=True).mean() 
    txx_stab_spamean= weighted_area_lat(txx_stab).mean('lat').mean('lon').tasmax
    slope, intercept, r_value, p_value, std_err = stats.linregress((txx_stab_spamean-ref1960_mean).time.dt.year.values, (txx_stab_spamean-ref1960_mean).values)
    print(f"\033[1mEUROPE\033[0m")
    print(f"NZ{year} mean: {np.round((txx_stab_spamean-ref1960_mean).mean('time').values, 1)} °C")
    print(f"NZ{year} std: {np.round((txx_stab_spamean-ref1960_mean).std('time').values, 1)} °C")
    print(f"NZ{year} trend: {np.round(slope, 4)} °C/year")
    print("")
    reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
    for reg in reg_eu:
        txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
        histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
        ref30y_hist = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
        
        txx = txx_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
        txx_spamean= weighted_area_lat(txx).mean('lat').mean('lon').tasmax
        slope, intercept, r_value, p_value, std_err = stats.linregress((txx_spamean-ref30y_hist).time.dt.year.values, 
                                                                       (txx_spamean-ref30y_hist).values)
        print(f"\033[1m{reg}\033[0m")
        print(f"NZ{year} mean: {np.round((txx_spamean-ref30y_hist).mean('time').values, 1)} °C")
        print(f"NZ{year} std: {np.round((txx_spamean-ref30y_hist).std('time').values, 1)} °C")
        print(f"NZ{year} trend: {np.round(slope, 4)} °C/year")
        print("")
    print("")
    print("")

## RR/FAR for ERA record (2)

RR

In [ ]:
rolmean=1
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
for i, year in enumerate(txx_stab.exp.values):
    thr = txxanom_eurmean_eramax
    count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
    count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
    p_0 = count_above/count_total

    count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
    count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
    p_1 = count_above/count_total
    if p_0 == 0:
        RR = np.nan
    else:
        RR = p_1/p_0
    print(f"EUROPE RR at {np.round(thr.values, 2)} for NZ{year}: {np.round(RR,1)}")

reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for reg in reg_eu:
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
    maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
    for i, year in enumerate(txx_stab.exp.values):
        thr = txxanom_eurmean_eramax
        count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
        count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
        p_0 = count_above/count_total
    
        count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
        count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
        p_1 = count_above/count_total
        if p_0 == 0:
            RR = np.nan
        else:
            RR = p_1/p_0
        print(f"{reg} RR at {np.round(thr.values, 2)} for NZ{year}: {np.round(RR,1)}")

FAR

In [ ]:
rolmean=1
txx_hist = txx_year_hist_eur
txx_ssp585 = txx_year_ssp585_eur
txx_era = txx_year_era_eur
txx_stab = txx_year_stab_eur

histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax

txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb

eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))

txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
for i, year in enumerate(txx_stab.exp.values):
    thr = txxanom_eurmean_eramax
    count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
    count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
    p_0 = count_above/count_total

    count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
    count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
    p_1 = count_above/count_total
    if p_1 == 0:
        FAR = np.nan
    else:
        FAR = 1-(p_0/p_1)
    print(f"EUROPE FAR at {np.round(thr.values, 2)} for NZ{year}: {np.round(FAR,2)}")

reg_eu = ['NEU', 'WCE', 'EEU', 'MED']
for reg in reg_eu:
    txx_hist = txx_year_hist.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_ssp585 = txx_year_ssp585.where(masks_scen_eur_dict[reg], drop=True).where(mask_scen_land==0, drop=True)
    txx_era = txx_year_era.where(masks_era_eur_dict[reg], drop=True).where(mask_era_land==0, drop=True)
    txx_stab = txx_year_stab.where(masks_stab_eur_dict[reg], drop=True).where(mask_stab_land==0, drop=True)
    
    histref1960 = txx_hist.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_hist_ensmean = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('realiz').mean('time').tasmax
    ref30y_hist_ensmemb = weighted_area_lat(histref1960).mean('lat').mean('lon').mean('time').tasmax
    
    txx_histssp585 = xr.concat([txx_hist, txx_ssp585], dim='time').rolling(time=rolmean, center=True).mean()
    txx_histssp585 = txx_histssp585.sel(time=slice(cftime.DatetimeProlepticGregorian(1995, 1, 1),cftime.DatetimeProlepticGregorian(2024, 12, 31)))
    txxanom_eurmean_histssp585 = weighted_area_lat(txx_histssp585).mean('lat').mean('lon').tasmax - ref30y_hist_ensmemb
    
    eraref1960  = txx_era.sel(time=slice(cftime.DatetimeProlepticGregorian(1961, 1, 1),cftime.DatetimeProlepticGregorian(1990, 12, 31)))
    ref30y_era = weighted_area_lat(eraref1960).mean('lat').mean('lon').mean('time').tasmax
    txxanom_eurmean_era = weighted_area_lat(txx_era.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_era
    txxanom_eurmean_eramax = txxanom_eurmean_era.sel(time=txxanom_eurmean_era.idxmax('time'))
    
    txxanom_eurmean_stab = weighted_area_lat(txx_stab.rolling(time=rolmean, center=True).mean()).mean('lat').mean('lon').tasmax-ref30y_hist_ensmean
    minvalue = txxanom_eurmean_stab.sel(exp=2030).sel(time=txxanom_eurmean_stab.sel(exp=2030).idxmin('time'))
    maxvalue = txxanom_eurmean_stab.sel(exp=2060).sel(time=txxanom_eurmean_stab.sel(exp=2060).idxmax('time'))
    for i, year in enumerate(txx_stab.exp.values):
        thr = txxanom_eurmean_eramax
        count_above = (txxanom_eurmean_histssp585 > thr).sum().item()
        count_total = (txxanom_eurmean_histssp585 >= txxanom_eurmean_histssp585.min().item()).sum().item()
        p_0 = count_above/count_total
    
        count_above = (txxanom_eurmean_stab.sel(exp=year) > thr).sum().item()
        count_total = (txxanom_eurmean_stab.sel(exp=year) >= txxanom_eurmean_stab.sel(exp=year).min().item()).sum().item()
        p_1 = count_above/count_total
        if p_1 == 0:
            FAR = np.nan
        else:
            FAR = 1-(p_0/p_1)
        print(f"{reg} FAR at {np.round(thr.values, 2)} for NZ{year}: {np.round(FAR,2)}")